<a href="https://colab.research.google.com/github/hyeonwooCH/Final_physiognomy_palmistry/blob/main/Algorithm_by_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. 경로 설정
import os
import glob

test_base_path = "/content/drive/MyDrive/c. Final_Team/Split_dataset/test"
test_images_path = os.path.join(test_base_path, "images")
test_masks_path = os.path.join(test_base_path, "masks")

# 눈썹 알고리즘
- **Glabella**: 미간
- **Length Ratio**: 눈썹 길이 / 눈 길이
- **Tilt**: 눈썹 시작 -> 눈썹 끝 각도
- **Palace Dist**: 눈썹 제일 아래부터 눈 제일 위 거리

In [ ]:
import os
import glob
import numpy as np
import cv2
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 1. 경로 설정
test_base_path = "/content/drive/MyDrive/c. Final_Team/Split_dataset/test"
test_images_path = os.path.join(test_base_path, "images")
test_masks_path = os.path.join(test_base_path, "masks")

# 2. 분석 알고리즘 함수
def analyze_eyebrow_physiognomy_gt(image_path, mask_path):
    # 이미지 및 마스크 로드
    img = Image.open(image_path).convert('RGB').resize((512, 512))
    mask = Image.open(mask_path).convert('L').resize((512, 512), Image.NEAREST)

    img_np = np.array(img)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    mask_np = np.array(mask)

    # [중요] AI Hub 라벨을 SegFace 라벨로 매핑 (기존 매핑 테이블 활용)
    # 2: lr(왼쪽 눈썹), 3: rr(오른쪽 눈썹), 4: le(왼쪽 눈), 5: re(오른쪽 눈)
    # 이미 매핑이 완료된 마스크라면 이 과정은 생략하거나 본인 라벨에 맞춰 수정하세요.
    l_eb_mask = (mask_np == 2).astype(np.uint8)
    r_eb_mask = (mask_np == 3).astype(np.uint8)
    l_eye_mask = (mask_np == 4).astype(np.uint8)
    r_eye_mask = (mask_np == 5).astype(np.uint8)

    def get_bbox_info(m):
        coords = np.column_stack(np.where(m > 0))
        if len(coords) == 0: return None
        return {'min_y': np.min(coords[:, 0]), 'min_x': np.min(coords[:, 1]),
                'max_y': np.max(coords[:, 0]), 'max_x': np.max(coords[:, 1]),
                'coords': coords}

    l_eb = get_bbox_info(l_eb_mask)
    r_eb = get_bbox_info(r_eb_mask)
    l_eye = get_bbox_info(l_eye_mask)

    analysis = {}
    if l_eb and r_eb and l_eye:
        # 1. 미간 너비 (인당): 오른쪽 눈썹 시작 - 왼쪽 눈썹 끝
        analysis['glabella_width'] = r_eb['min_x'] - l_eb['max_x']

        # 2. 눈썹 길이 비율: 눈썹 가로 / 눈 가로 (1.2 이상이면 장미)
        eye_w = l_eye['max_x'] - l_eye['min_x']
        eb_w = l_eb['max_x'] - l_eb['min_x']
        analysis['length_ratio'] = eb_w / eye_w

        # 3. 눈썹 기울기: 앞머리와 꼬리의 y좌표 차이로 각도 계산
        pts = l_eb['coords']
        head = pts[np.argmin(pts[:, 1])] # 가장 왼쪽 점
        tail = pts[np.argmax(pts[:, 1])] # 가장 오른쪽 점
        # 관상학적 각도 (Degree)
        analysis['tilt'] = np.degrees(np.arctan2(head[0] - tail[0], tail[1] - head[1]))

        # 4. 전택궁 (눈과 눈썹 사이 거리): 눈썹 하단 - 눈 상단
        analysis['palace_dist'] = l_eye['min_y'] - l_eb['max_y']

    return img_np, mask_np, analysis

# 3. 실행 및 시각화
img_files = sorted(glob.glob(os.path.join(test_images_path, "*.*")))
mask_files = sorted(glob.glob(os.path.join(test_masks_path, "*.*")))

# 샘플 하나 분석
idx = 0
img_viz, mask_viz, res = analyze_eyebrow_physiognomy_gt(img_files[idx], mask_files[idx])

# 시각화
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.imshow(img_viz)
plt.title(f"Original: {os.path.basename(img_files[idx])}")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_viz)
# 눈썹(2,3)과 눈(4,5) 영역만 하이라이트
overlay = np.isin(mask_viz, [2, 3, 5, 4])
plt.imshow(overlay, cmap='spring', alpha=0.5)

# 분석 결과 텍스트 오버레이
info_text = (f"Glabella: {res['glabella_width']}px\n"
             f"Length Ratio: {res['length_ratio']:.2f}\n"
             f"Tilt: {res['tilt']:.1f}°\n"
             f"Palace Dist: {res['palace_dist']}px")
plt.text(10, 80, info_text, color='white', fontsize=12, fontweight='bold',
         bbox=dict(facecolor='black', alpha=0.6))

plt.title("Eyebrow Physiognomy Analysis (GT)")
plt.axis('off')
plt.show()

# 이마 알고리즘
- **Forehead ratio**: 이마 비율
- **Hairline**: M자형 판별
- **Symmetry**: 대칭성
- **Center Bright**: 중앙 돌출도

In [ ]:
import os
import glob
import numpy as np
import cv2
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# 1. 경로 설정 (기존과 동일)
test_base_path = "/content/drive/MyDrive/c. Final_Team/Split_dataset/test"
test_images_path = os.path.join(test_base_path, "images")
test_masks_path = os.path.join(test_base_path, "masks")

# 2. 이마 분석 알고리즘 함수
def analyze_forehead_physiognomy_gt(image_path, mask_path):
    # 이미지 및 마스크 로드
    img = Image.open(image_path).convert('RGB').resize((512, 512))
    mask = Image.open(mask_path).convert('L').resize((512, 512), Image.NEAREST)

    img_np = np.array(img)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    mask_np = np.array(mask)

    # [라벨 정의] AI Hub/SegFace 매핑 기준 (본인 데이터셋 라벨 확인 필수)
    # 일반적인 기준: Skin=1, Hair=17, L_brow=2, R_brow=3
    skin_mask = (mask_np == 1).astype(np.uint8)
    hair_mask = (mask_np == 17).astype(np.uint8)
    brow_mask = np.isin(mask_np, [2, 3]).astype(np.uint8)

    # 좌표 추출 함수
    def get_coords(m):
        return np.column_stack(np.where(m > 0))

    skin_pts = get_coords(skin_mask)
    brow_pts = get_coords(brow_mask)

    analysis = {}
    if len(skin_pts) > 0 and len(brow_pts) > 0:
        # --- 영역 정의 ---
        face_top_y = np.min(skin_pts[:, 0])    # 헤어라인 최상단 (발제)
        face_bottom_y = np.max(skin_pts[:, 0]) # 턱 끝
        brow_top_y = np.min(brow_pts[:, 0])    # 눈썹 상단 라인

        # 1. 이마 비율 (H_forehead / H_face)
        forehead_h = brow_top_y - face_top_y
        face_h = face_bottom_y - face_top_y
        analysis['forehead_ratio'] = forehead_h / face_h

        # 2. 발제선(Hairline) 형태 분석
        # 이마 상단 경계선(Hair와 Skin이 만나는 지점) 샘플링
        hairline_pts = []
        x_start, x_end = np.min(skin_pts[:, 1]), np.max(skin_pts[:, 1])
        for x in range(x_start + 20, x_end - 20, 5):
            y_vals = np.where(skin_mask[:brow_top_y, x] > 0)[0]
            if len(y_vals) > 0:
                hairline_pts.append(y_vals[0])

        if len(hairline_pts) > 10:
            # M자형 판별: 양 끝 평균 y 대비 중앙 y가 얼마나 깊은지(큰지) 확인
            edge_avg = (hairline_pts[0] + hairline_pts[-1]) / 2
            mid_val = hairline_pts[len(hairline_pts)//2]
            # 중앙이 양끝보다 15px 이상 아래로 내려오면 M자형으로 간주 (임계값 조정 가능)
            analysis['is_m_shape'] = mid_val > edge_avg + 15
        else:
            analysis['is_m_shape'] = False

        # 3. 대칭성 (Symmetry)
        mid_x = (x_start + x_end) // 2
        left_area = np.sum(skin_mask[:brow_top_y, x_start:mid_x])
        right_area = np.sum(skin_mask[:brow_top_y, mid_x:x_end])
        analysis['symmetry'] = min(left_area, right_area) / max(left_area, right_area)

        # 4. 중앙 돌출도 (Brightness 기반 추정)
        center_y = (face_top_y + brow_top_y) // 2
        center_roi = gray[center_y-20:center_y+20, mid_x-20:mid_x+20]
        analysis['center_brightness'] = np.mean(center_roi) if center_roi.size > 0 else 0

    return img_np, mask_np, analysis

# 3. 실행 및 시각화
img_files = sorted(glob.glob(os.path.join(test_images_path, "*.*")))
mask_files = sorted(glob.glob(os.path.join(test_masks_path, "*.*")))

idx = 0
img_viz, mask_viz, res = analyze_forehead_physiognomy_gt(img_files[idx], mask_files[idx])

# 시각화 부분
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.imshow(img_viz)
plt.title(f"Original: {os.path.basename(img_files[idx])}")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_viz)

# 이마 영역 시각화 (Skin 중 눈썹 위 영역만 하이라이트)
brow_y_limit = np.min(np.where(np.isin(mask_viz, [2, 3]))[0])
forehead_overlay = (mask_viz == 1) & (np.indices((5 12, 512))[0] < brow_y_limit)
plt.imshow(forehead_overlay, cmap='cool', alpha=0.4)

# 분석 결과 텍스트
m_text = "M-Shape" if res['is_m_shape'] else "Round/Straight"
info_text = (f"Forehead Ratio: {res['forehead_ratio']:.2f}\n"
             f"Hairline: {m_text}\n"
             f"Symmetry: {res['symmetry']:.2f}\n"
             f"Center Bright: {res['center_brightness']:.1f}")

plt.text(10, 80, info_text, color='white', fontsize=12, fontweight='bold',
         bbox=dict(facecolor='black', alpha=0.6))

plt.title("Forehead Physiognomy Analysis (GT)")
plt.axis('off')
plt.show()

# 눈 알고리즘

In [ ]:
import numpy as np
import cv2
import torch
from PIL import Image
import matplotlib.pyplot as plt

def analyze_eye_physiognomy_gt(image_path, mask_path):
    # 1. 이미지 및 마스크 로드 (512x512)
    img = Image.open(image_path).convert('RGB').resize((512, 512))
    mask = Image.open(mask_path).convert('L').resize((512, 512), Image.NEAREST)

    img_np = np.array(img)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    mask_np = np.array(mask)

    # [라벨 정의] 4: le, 5: re (눈 영역)
    # 눈동자는 별도의 임계값 처리가 필요할 수 있으나, 여기서는 눈 영역 내 어두운 부분으로 계산
    l_eye_mask = (mask_np == 4).astype(np.uint8)

    analysis = {}

    def get_eye_details(eye_mask):
        coords = np.column_stack(np.where(eye_mask > 0))
        if len(coords) < 10: return None

        ymin, xmin, ymax, xmax = np.min(coords[:, 0]), np.min(coords[:, 1]), np.max(coords[:, 0]), np.max(coords[:, 1])
        w = xmax - xmin
        h = ymax - ymin

        # --- 1단계: 기하학적 구조 ---
        l_ratio = w / h  # 가로세로비

        # 눈꼬리 각도 (앞머리 대비 꼬리의 높낮이)
        eye_pts = np.column_stack(np.where(eye_mask > 0))
        head_pt = eye_pts[np.argmin(eye_pts[:, 1])]
        tail_pt = eye_pts[np.argmax(eye_pts[:, 1])]
        theta_tail = np.degrees(np.arctan2(head_pt[0] - tail_pt[0], tail_pt[1] - head_pt[1]))

        # --- 2단계: 흑백 및 안광 분석 ---
        eye_roi = gray[ymin:ymax, xmin:xmax]
        # 단순 이진화로 검은자위(Iris) 추정
        _, iris_bin = cv2.threshold(eye_roi, np.percentile(eye_roi, 30), 255, cv2.THRESH_BINARY_INV)
        r_iris = np.sum(iris_bin/255) / (w * h)

        # 안광(Specular Highlight) 추출: 매우 밝은 점(240 이상)
        _, highlight_bin = cv2.threshold(eye_roi, 240, 255, cv2.THRESH_BINARY)
        specular_strong = np.sum(highlight_bin) > 0

        # --- 3단계: 수분감 (Reflectance) ---
        # 눈 주변의 평균 밝기와 표준편차로 습함/건조함 추정
        std_val = np.std(eye_roi) # 표준편차가 낮고 밝기가 일정하면 습한(Watery) 느낌
        is_watery = std_val < 15 and np.mean(eye_roi) > 100

        return {
            "l_ratio": l_ratio,
            "theta_tail": theta_tail,
            "r_iris": r_iris,
            "specular": specular_strong,
            "is_watery": is_watery,
            "bbox": [ymin, xmin, ymax, xmax]
        }

    analysis['left_eye'] = get_eye_details(l_eye_mask)
    return img_np, mask_np, analysis

# --- 실행 및 결과 출력 ---
img_files = sorted(glob.glob(os.path.join(test_images_path, "*.*")))
mask_files = sorted(glob.glob(os.path.join(test_masks_path, "*.*")))

img_viz, mask_viz, res = analyze_eye_physiognomy_gt(img_files[195], mask_files[195])
eye = res['left_eye']

if eye:
    print(f"--- 눈 관상 분석 결과 ---")
    print(f"1. 구조: 길이비 {eye['l_ratio']:.2f} ({'가늘고 긴 눈' if eye['l_ratio'] > 2.5 else '둥근 눈'})")
    print(f"   각도: {eye['theta_tail']:.1f}° ({'치켜올라감' if eye['theta_tail'] > 5 else '평탄/처짐'})")
    print(f"2. 안광: {'강렬함(안광 있음)' if eye['specular'] else '안광 없음(흐릿함)'}")
    print(f"3. 특징: {'도화안(습함)' if eye['is_watery'] else '맑고 건조함'}")

# 시각화 (눈 영역 확대)
ymin, xmin, ymax, xmax = eye['bbox']
plt.imshow(img_viz[ymin-20:ymax+20, xmin-20:xmax+20])
plt.title("Eye Detailed Analysis")
plt.axis('off')
plt.show()